In [ ]:
# ! pip install imutils

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from PIL import Image
import tensorflow as tf
import cv2
import seaborn as sns
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
import os
from tqdm import tqdm
import imutils
import keras
from keras.applications import resnet
from keras import backend as K
from keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,BatchNormalization,Activation,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import imagenet_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import matplotlib.pyplot as plt
IMG_SIZE = 256
training = '/kaggle/input/brain-tumor-mri-dataset/Training'
testing = '/kaggle/input/brain-tumor-mri-dataset/Testing'

In [ ]:
# def crop_img(img):
#     gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     gray = cv2.GaussianBlur(gray, (3, 3), 0)

#     # threshold the image, then perform a series of erosions +
#     # dilations to remove any small regions of noise
#     thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
#     thresh = cv2.erode(thresh, None, iterations=2)
#     thresh = cv2.dilate(thresh, None, iterations=2)

#     # find contours in thresholded image, then grab the largest one
#     cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     c = max(cnts, key=cv2.contourArea)

#     # find the extreme points
#     extLeft = tuple(c[c[:, :, 0].argmin()][0])
#     extRight = tuple(c[c[:, :, 0].argmax()][0])
#     extTop = tuple(c[c[:, :, 1].argmin()][0])
#     extBot = tuple(c[c[:, :, 1].argmax()][0])
#     ADD_PIXELS = 0
#     new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

#     return new_img

In [ ]:
# training_dir = os.listdir(training)
# testing_dir = os.listdir(testing)
# trainData = []
# trainLabels = []
# label = 0

# for dir in training_dir:
#     save_path = 'cleaned/Training/'+ dir
#     path = os.path.join(training,dir)
#     image_dir = os.listdir(path)
#     for img in image_dir:
#         image = cv2.imread(os.path.join(path,img))
#         new_img = crop_img(image)
#         new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
#         if not os.path.exists(save_path):
#             os.makedirs(save_path)
#         cv2.imwrite(save_path+'/'+img, new_img)

# for dir in testing_dir:
#     save_path = 'cleaned/Testing/'+ dir
#     path = os.path.join(testing,dir)
#     image_dir = os.listdir(path)
#     for img in image_dir:
#         image = cv2.imread(os.path.join(path,img))
#         new_img = crop_img(image)
#         new_img = cv2.resize(new_img,(IMG_SIZE,IMG_SIZE))
#         if not os.path.exists(save_path):
#             os.makedirs(save_path)
#         cv2.imwrite(save_path+'/'+img, new_img)



In [ ]:
trainData=[]
trainLabels=[]

for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Training/notumor'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'notumor' in image_path:
        label = 0
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Training/glioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'glioma' in image_path:
        label = 1
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Training/pituitary'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'pituitary' in image_path:
        label = 2
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Training/meningioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'meningioma' in image_path:
        label = 3
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        trainData.append(image_array)
        trainLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")

In [ ]:
# trainData = np.array(trainData)
# trainData = trainData/255.0
# print(trainData.shape)

In [ ]:
testData=[]
testLabels=[]

for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Testing/notumor'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'notumor' in image_path:
        label = 0
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Testing/glioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'glioma' in image_path:
        label = 1
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'pituitary' in image_path:
        label = 2
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")
for root, _, files in os.walk('/kaggle/input/brain-tumor-mri-dataset/Testing/meningioma'):
  for file in files:
    if file.endswith('.jpg'):
      image_path = os.path.join(root,file)
      if 'meningioma' in image_path:
        label = 3
        image = cv2.imread(image_path)
        image = cv2.resize(image,(224,224))
        image_array = np.array(image)
        testData.append(image_array)
        testLabels.append(label)
        #print(f"Image Path: {image_path}, Label: {label}")

In [ ]:
# testData = np.array(testData)
# testData = testData/255.0
# print(testData.shape)

In [ ]:
# testLabels = np.array(testLabels)
# print(testLabels.shape)

In [ ]:
# trainLabels = np.array(trainLabels)
# print(trainLabels.shape)

In [ ]:
# valData, testData = train_test_split(testData, train_size=.5, random_state=20)

In [ ]:
# valLabels, testLabels = train_test_split(testLabels, train_size=.5, random_state=20)
# print(valLabels.shape)
# print(testLabels.shape)
# print(valData.shape)
# print(testData.shape)

In [ ]:
from keras.utils import to_categorical
X_train = trainData
y_train = trainLabels
X_test = testData
y_test = testLabels

y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

X_val, X_test = train_test_split(X_test, train_size=.3, random_state=20)
y_val, y_test = train_test_split(y_test, train_size=.3, random_state=20)

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)
X_train = np.array(X_train)
X_test = np.array(X_test)
X_val = np.array(X_val)


print(f"Shape of images in X_train: {X_train.shape}")
print(f"Shape of images in X_test: {X_test.shape}")
print(f"Shape of images in X_val: {X_val.shape}")
print(f"Shape of images in y_train: {y_train.shape}")
print(f"Shape of images in y_test: {y_test.shape}")
print(f"Shape of images in y_val: {y_val.shape}")

In [ ]:
model = resnet.ResNet50(
      input_shape = (224, 224, 3),
      include_top = False,
      weights = 'imagenet'
    )
model.summary()

In [ ]:
for layers in model.layers:
    layers.trainable = False

topLayer = Flatten()(model.output)
topLayer = Dropout(0.5)(topLayer)
topLayer = Dense(4, activation = "softmax")(topLayer)
model = keras.Model(model.input, topLayer)

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [
    'accuracy',
    tf.keras.metrics.AUC(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.F1Score()])
model.summary()

In [ ]:
ESM = tf.keras.callbacks.EarlyStopping(patience=4, monitor='accuracy')
rn = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30, verbose=1, callbacks=ESM)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
accuracy = rn.history['loss']
validation_accuracy = rn.history['val_loss']

epochs = range(len(accuracy))

plt.plot(epochs, accuracy, 'r', label='Training Accuracy')
plt.plot(epochs,  validation_accuracy, 'g', label="Validation Accuracy")
plt.legend(loc=0)

plt.show()

In [ ]:
pred = model.predict(X_test)
classPred = np.argmax(pred, axis=1)
classLabel = np.argmax(y_test, axis=1)
con_matrix = confusion_matrix(classLabel, classPred)

plt.figure(figsize=(8, 6))
sns.heatmap(con_matrix, annot=True, fmt='d', cmap='Reds',
            xticklabels=['notumor','glioma', 'pituitary' ,'meningioma'],
            yticklabels=['notumor','glioma', 'pituitary' ,'meningioma'])

plt.xlabel('Predictions')
plt.ylabel('Real')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
mobileModel = keras.applications.MobileNet(input_shape=(224,224,3), include_top=False)
newModel = Sequential()
newModel.add(mobileModel)
newModel.add(Flatten())
newModel.add(Dropout(0.5))
newModel.add(Dense(4, activation="softmax"))

newModel.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = [
    'accuracy',
    tf.keras.metrics.AUC(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.Precision(),
    tf.keras.metrics.F1Score()])

newModel.summary()


In [ ]:
newModel.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=1, callbacks=ESM)

In [ ]:
newModel.evaluate(X_test, y_test)

In [ ]:
os.chdir("/kaggle/input/brain-tumor-mri-dataset/Testing")
dict_out = {}
test_sub_dirs = os.listdir()[::-1]
for i in range(len(test_sub_dirs)):
    dict_out.update({test_sub_dirs[i]: i})


actual_classes = [list(dict_out.keys())[list(dict_out.values()).index(val)] for val in classLabel]

fig, axes = plt.subplots(4, 5, figsize=(15, 15))

for i in range(20):
    axes[i//5, i%5].imshow(X_test[i])
    axes[i//5, i%5].set_title(f"Predicted: {classLabel[i]} \n Actual: {actual_classes[i]}")
plt.show()